In [1]:
from dataclasses import dataclass
import re


@dataclass  # This decorator helps save time from initializing constructor
class Flash_card:
    """This contains a question, its demonstration, possible choices, and correct answers out of the choices"""

    question: str
    choices: list
    correct_choices: list
    demo: str = None

    # Shuffle all choices
    def shuffle_choices(self):
        np.random.shuffle(self.choices)

In [2]:
from termcolor import colored
import numpy as np


class Folder:
    """This contains multiple flash cards"""

    my_flash = list()
    correct: int = 0

    def total_score(self):
        """This prints the percentage of correct answers"""
        try:
            print(f"You get {self.correct * 100 / len(self.my_flash):.2f}%")
        except:
            print("You have to have at least one question")

    def learn(self):
        """The user chooses correct answer(s) by typing a, b, c, etc. (lower case) and hit enter"""
        # Shuffle order of questions
        np.random.shuffle(self.my_flash)
        for i, flash in enumerate(self.my_flash, 1):
            print(colored(f"Q{i}. {flash.question}", "blue"))  # Print question
            if flash.demo:
                print(flash.demo.strip())  # Print question demonstration
            # Shuffle choices
            flash.shuffle_choices()
            for index, c in enumerate(flash.choices):
                print(
                    colored(f"{chr(index + 97)}. {c}", "magenta")
                )  # Print possible choices
            answers = []
            if len(flash.correct_choices) > 1:
                answer = input(
                    "Choose muitiple answers as followings: a(space)b(space)c "
                )
                for a in answer.split():
                    answers.append(flash.choices[ord(a) - 97])
            else:
                ans = input("Choose one correct answer")
                answers.append(flash.choices[ord(ans) - 97])
            if sorted(answers) == sorted(flash.correct_choices):  # Correct answer(s)
                print("Correct!")
                self.correct += 1
            else:  # Incorrect answer(s)
                print(colored(f"Wrong! Correct choice should be: ", "yellow"), end="")
                print(colored("\n".join(flash.correct_choices), "red"))
            print()
        self.total_score()

In [3]:
def read_a_file(file_path: str, general_choice_pattern: str, correct_choice_pattern: str) -> Folder:
    """This reads a text file and returns a Folder instance"""
    doc = Folder()
    with open(file_path, "r") as f:
        q, demo, correct_choices, choices = "", "", [], []
        for line in f:
            correct = re.match(correct_choice_pattern, line)
            regular = re.match(general_choice_pattern, line)
            # print(f'line: {line}')
            # Read multiple-lined questions
            if not regular:
                q += line
                # print(f'not regular, q: {q}')
            else:
                # print(f'regular, neat q: {q}')
                if correct:  # Get correct choices
                    correct_choice = line[correct.end():].strip()
                    correct_choices.append(correct_choice)
                    # Any way, the regular choice is added
                    choices.append(correct_choice)
                    # print(f'correct_choice: {correct_choice}')
                else:  # Get incorrect choices
                    regular_choice = line[regular.end():].strip()
                    # print(f'incorrect_choice: {regular_choice}')
                    choices.append(regular_choice)
            if line in ['\n', '\r\n']:
                # Extract the question
                # print(f'q before split: {q}')
                q = q.split(' ', 1)[-1]
                # print(f'q after split: {q}')
                q = q.replace('\n', '').strip()
                # print(f'q after replace: {q}')
                a_flash_card = Flash_card(
                    question=q,
                    choices=choices,
                    correct_choices=correct_choices,
                )
                doc.my_flash.append(a_flash_card)
                # print(f'question: {q}')
                # print(f'answers: {choices}')
                # print(f'correct answer: {correct_choices}')
                # print()
                q, demo, correct_choices, choices = "", "", [], []
    return doc

In [4]:
def create_a_nicer_file(doc: Folder, filename: str):
    """
    This is to create a nicer Q&A file name from doc a Folder instance
    """
    with open(str(filename), "w") as f:
        for i, flash in enumerate(doc.my_flash, 1):
            f.write(f"####Q{i}. {flash.question}\n")  # Write question
            if flash.demo:  # Write demo if it exists
                f.write(flash.demo)
            for choice in flash.choices:
                if choice in flash.correct_choices:
                    f.write(f"- [x] {choice}\n")
                else:
                    f.write(f"- [ ] {choice}\n")
            f.write("\n")

In [ ]:
file_name = "5215.txt"
doc = read_a_file(file_name, general_choice_pattern=r'^\s*-\s*\[\s*\w*\s*\]\s*', correct_choice_pattern=r'^\s*-\s*\[\s*\w\s*\]\s*')
create_a_nicer_file(doc, file_name.split(".", 1)[0] + ".txt")
doc.learn()

Q1. The Q in Q-learning for reinforcement learning is best described as
a. The discount factor
b. The sum of future expected rewards
c. The reward signal from the environment
d. the reward prediction error quotient


Choose one correct answer b


Correct!

Q2. Check the scenario where density-based clustering algorithms like DBSCAN are expected to outperform K-means
a. When the clusters would share the same center-point, such as two concentric circles
b. When the clusters are well represented by spheres
c. When we know exactly how many clusters to expect


Choose one correct answer a


Correct!

Q3. Two child nodes of the same parent are independent until the parent node is observed, which then introduces a dependency.
a. False
b. True


Choose one correct answer b


Wrong! Correct choice should be: False

Q4. Assuming the value of B is observed ("known"), in which of the graphs would the value of node A now depend upon the value of node C.
a. A <-- B --> C
b. A --> B <-- C
c. A --> B --> C
d. A <-- B <-- C
